[View in Colaboratory](https://colab.research.google.com/github/maddy46/mlblr4a_experiments/blob/master/DNST_CIFAR10_AUG_2905.ipynb)

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [53]:
!rm utils.py
!wget https://raw.githubusercontent.com/Zahlii/colab-tf-utils/master/utils.py
import utils
import os
import keras

def compare(best, new):
  return best.losses['val_acc'] < new.losses['val_acc']

def path(new):
  if new.losses['val_acc'] > 0.2:
    return '4B_008_%s.h5' % new.losses['val_acc']

callbacks = cb = [
      utils.GDriveCheckpointer(compare,path)
]

--2018-05-28 16:14:30--  https://raw.githubusercontent.com/Zahlii/colab-tf-utils/master/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6935 (6.8K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   6.77K  --.-KB/s    in 0s      

2018-05-28 16:14:30 (71.5 MB/s) - ‘utils.py’ saved [6935/6935]



In [0]:
# Hyperparameters
num_classes = 10
epochs =50
#l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [55]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print (img_height)
print (img_width)

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

32
32


In [56]:
print (x_train.shape[0])
print (x_test.shape[0])

50000
10000


In [57]:
'''
new_images = np.empty((50000, 32, 32, 3), dtype='uint8')
new_test = np.empty((10000, 32, 32, 3), dtype='uint8')

for image_count in range(x_train.shape[0]):
    new_images[image_count] = cv2.resize(x_train[image_count], (32, 32))
for image_count1 in range(x_test.shape[0]):
    new_test[image_count1] = cv2.resize(x_test[image_count1], (32, 32))
img_height1, img_width1, channel1 = new_images.shape[1],new_images.shape[2],new_images.shape[3]
print (img_height1)
print (img_width1)

print('x resizing Done.')
'''

32
32
x Processing Done.


In [0]:
datagen = ImageDataGenerator(zoom_range=0.2, rotation_range=20)
datagen.fit(x_train)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [62]:
num_filter = 16
dropout_rate = 0.2
l = 6
batch_size = 32
epochs =50
input = Input(shape=(img_height, img_width, channel,))
print(channel)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)
#output = output_layer(First_Block)


3


In [63]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_216 (Conv2D)             (None, 32, 32, 16)   432         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_216 (BatchN (None, 32, 32, 16)   64          conv2d_216[0][0]                 
__________________________________________________________________________________________________
activation_214 (Activation)     (None, 32, 32, 16)   0           batch_normalization_216[0][0]    
__________________________________________________________________________________________________
conv2d_217

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
'''
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),callbacks=cb)
                    '''

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, 
                    verbose=1, validation_data=(x_test
                                                , y_test),
                    steps_per_epoch=len(x_train)/batch_size, callbacks=cb)

Epoch 1/50
1562/1562 [============================>.] - ETA: 0s - loss: 1.6986 - acc: 0.3719

1563/1562 [==============================] - 2242s 1s/step - loss: 1.6985 - acc: 0.3719 - val_loss: 1.9348 - val_acc: 0.3780


Uploading file 4B_008_0.378.h5 to folder Colab Notebooks: 100%|██████████| 100/100 [00:01<00:00, 98.91it/s]


Epoch 2/50
1562/1562 [============================>.] - ETA: 0s - loss: 1.4333 - acc: 0.4804

1563/1562 [==============================] - 2218s 1s/step - loss: 1.4332 - acc: 0.4805 - val_loss: 1.4034 - val_acc: 0.5096
Removing old cloud file 4B_008_0.378.h5


...

1563/1562 [==============================] - 2226s 1s/step - loss: 1.0379 - acc: 0.6295 - val_loss: 1.3462 - val_acc: 0.5943
No improvement.
Epoch 10/50
 312/1562 [====>.........................] - ETA: 27:45 - loss: 1.0052 - acc: 0.6456

1562/1562 [============================>.] - ETA: 0s - loss: 1.0123 - acc: 0.6408

1563/1562 [==============================] - 2228s 1s/step - loss: 1.0123 - acc: 0.6408 - val_loss: 1.4139 - val_acc: 0.5768
No improvement.
Epoch 11/50
 312/1562 [====>.........................] - ETA: 27:36 - loss: 0.9946 - acc: 0.6416

 761/1562 [=============>................] - ETA: 17:43 - loss: 0.9987 - acc: 0.6410

KeyboardInterrupt: ignored

In [17]:
# Save the trained weights in to .h5 format
model.save_weights("saved_model.h5")
print("Saved model to disk")

Saved model to disk
